In [1]:
from glob import iglob
from datetime import datetime
from itertools import chain, repeat

import numpy as np
import pandas as pd
from cytoolz import compose, reduce
from h5py import File
from dask import delayed, compute
from dask.bag import from_sequence
from dask.diagnostics import ProgressBar

In [15]:
def read_afile(filename):
    try:
        with File(filename, 'r') as f:
            yield from ({'filename': filename,
                         'tag': tag,
                         'iom_intensity_pc': iom,
                         'img': img.astype('double'),
                         'img_intensity': img.sum(),
                         'delay': round(dt, 2)}
                        for tag, iom, img, dt
                        in zip(f['/bunches'],
                               f['/photon_diagnostics/FEL01'
                                 '/I0_monitor/iom_sh_a_pc'],
                               f['/vmi/andor'],
                               f['/user_laser/delay_line/position']))
    except Exception as err:
        print(err)
        yield from ()

In [16]:
filenames = sorted(iglob("/data/Test/Run_010/rawdata/*.h5"))
df = (
    from_sequence(filenames)
    .map(read_afile)
    .flatten()
    .to_dataframe()
)
df

,delay,filename,img,img_intensity,iom_intensity_pc,tag
npartitions=20,,,,,,
,float64,object,object,int64,float64,int64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [ ]:
bg_period, bg_mod = 3, 0  # /Background_Period
df['is_bg'] = df['tag'] % bg_period == bg_mod
delays = df['delay'].unique().compute()

with ProgressBar():
    summed = compute(
        [
            {
                'is_bg': bg, 'delay': dt,
                'count': delayed(df[(df['is_bg'] == bg) & (df['delay'] == dt)]['img'].to_bag().count()),
                'summed': delayed(df[(df['is_bg'] == bg) & (df['delay'] == dt)]['img'].to_bag().sum()),
            }
            for bg in [True, False] for dt in delays
        ],
    )

[###############                         ] | 38% Completed | 19.6s

In [17]:
summed

([{'is_bg': True,
   'delay': -5.6,
   'count': 666,
   'summed': array([[67860., 66220., 67226., ..., 67459., 67892., 67023.],
          [67008., 67633., 66791., ..., 67970., 66785., 66831.],
          [67777., 67004., 66512., ..., 67285., 66725., 67737.],
          ...,
          [67567., 67546., 67129., ..., 66646., 66691., 67105.],
          [68224., 66876., 67290., ..., 67691., 67224., 67732.],
          [67529., 67033., 66882., ..., 66877., 67731., 66865.]])},
  {'is_bg': False,
   'delay': -5.6,
   'count': 1334,
   'summed': array([[135791., 132824., 134841., ..., 135084., 136084., 134243.],
          [134041., 135427., 133505., ..., 136092., 133759., 134096.],
          [135746., 134591., 133408., ..., 134795., 133848., 135716.],
          ...,
          [135367., 135027., 134296., ..., 133164., 133566., 134169.],
          [136682., 134195., 134913., ..., 135685., 134537., 135570.],
          [135387., 134470., 134007., ..., 133996., 135531., 133851.]])}],)